<a href="https://colab.research.google.com/github/bheshaj96/tensorflow-exercises/blob/master/blstm_gsm_orig_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PLease note that, i have extracted mfcc for train and test separately. After 
extracting for train i scaled it as 
x_new = [x-mean(x)]/[max(x)-min(x)]
and stored the values of mean, max and min for each feature component. After that for test file i have used these stored values to scale test features. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/My Drive

/content/drive/My Drive


In [3]:
import sys
import numpy as np
import tensorflow as tf
import librosa as lr
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
#np.set_printoptions(threshold=sys.maxsize)

In [4]:
cur_dir = os.getcwd()
print(cur_dir)
orig_dir = os.path.join(cur_dir, 'iisc_7_july/original')
sip_dir = os.path.join(cur_dir, 'iisc_7_july/sip')
gsm_dir = os.path.join(cur_dir, 'iisc_7_july/gsm')
orig_filenames = os.listdir(orig_dir)
sip_filenames = os.listdir(sip_dir)
gsm_filenames = os.listdir(gsm_dir)

/content/drive/My Drive


In [5]:
num_files = 3000
mfcc_dim = 13

Here extracting train+val mfccs

In [6]:
## 2
gsm_unnormalised = []
orig_unnormalised= []

train_mfcc_nos = [0]
for orig_file in tqdm(orig_filenames):
  if orig_file in gsm_filenames:
    orig_audio_path = os.path.join(orig_dir, orig_file)
    gsm_audio_path = os.path.join(gsm_dir, orig_file)

    orig, orig_sr = lr.load(orig_audio_path)
    gsm, gsm_sr = lr.load(gsm_audio_path)

    if (len(orig)!=len(gsm)): 
        min_len = np.min((len(orig), len(gsm)))
        orig = orig[0:min_len-1]
        gsm = gsm[0:min_len-1]
    orig_mfcc = lr.feature.mfcc(orig, orig_sr, n_mfcc= mfcc_dim, hop_length=int(orig_sr/100), n_fft=int(orig_sr/40))  
    orig_mfcc = np.vstack((orig_mfcc,lr.feature.delta(orig_mfcc, order=1),lr.feature.delta(orig_mfcc,order= 2)))
    gsm_mfcc = lr.feature.mfcc(gsm, gsm_sr, n_mfcc= mfcc_dim, hop_length=int(orig_sr/100), n_fft=int(orig_sr/40))
    gsm_mfcc = np.vstack((gsm_mfcc,lr.feature.delta(gsm_mfcc, order=1),lr.feature.delta(gsm_mfcc,order= 2)))
    train_mfcc_nos.append(orig_mfcc.shape[1])
   
    gsm_unnormalised.append(gsm_mfcc.T)
    orig_unnormalised.append(orig_mfcc.T)

orig_unnormalised_stacked = np.vstack((np.array(orig_unnormalised)))
gsm_unnormalised_stacked = np.vstack((np.array(gsm_unnormalised)))
train_mfcc_nos = np.array(train_mfcc_nos)

Here extracting test mfccs

In [ ]:
## Test
gsm_unnormalised_test = []
orig_unnormalised_test = []

test_mfcc_nos = [0]
for orig_file in tqdm(orig_filenames[1800:2800]):
  if orig_file in gsm_filenames:
    orig_audio_path = os.path.join(orig_dir, orig_file)
    gsm_audio_path = os.path.join(gsm_dir, orig_file)

    orig, orig_sr = lr.load(orig_audio_path)
    gsm, gsm_sr = lr.load(gsm_audio_path)

    if (len(orig)!=len(gsm)): 
        min_len = np.min((len(orig), len(gsm)))
        orig = orig[0:min_len-1]
        gsm = gsm[0:min_len-1]
    orig_mfcc_test = lr.feature.mfcc(orig, orig_sr, n_mfcc= mfcc_dim, hop_length=int(orig_sr/100), n_fft=int(orig_sr/40))  
    orig_mfcc_test = np.vstack((orig_mfcc_test,lr.feature.delta(orig_mfcc_test, order=1),lr.feature.delta(orig_mfcc_test,order= 2)))
    gsm_mfcc_test = lr.feature.mfcc(gsm, gsm_sr, n_mfcc= mfcc_dim, hop_length=int(orig_sr/100), n_fft=int(orig_sr/40))
    gsm_mfcc_test = np.vstack((gsm_mfcc_test,lr.feature.delta(gsm_mfcc_test, order=1),lr.feature.delta(gsm_mfcc_test,order= 2)))
    test_mfcc_nos.append(orig_mfcc_test.shape[1])

    gsm_unnormalised_test.append(gsm_mfcc_test.T) 
    orig_unnormalised_test.append(orig_mfcc_test.T)

orig_unnormalised_stacked_test = np.vstack((np.array(orig_unnormalised_test)))
gsm_unnormalised_stacked_test = np.vstack((np.array(gsm_unnormalised_test)))
test_mfcc_nos = np.array(test_mfcc_nos)

Now performing feature scaling

In [7]:
orig_stacked = np.zeros((orig_unnormalised_stacked.shape[0],orig_unnormalised_stacked.shape[1]))
gsm_stacked = np.zeros((gsm_unnormalised_stacked.shape[0],gsm_unnormalised_stacked.shape[1]))
#orig_stacked_test = np.zeros((orig_unnormalised_stacked_test.shape[0],orig_unnormalised_stacked_test.shape[1]))
#gsm_stacked_test = np.zeros((gsm_unnormalised_stacked_test.shape[0],gsm_unnormalised_stacked_test.shape[1]))

orig_stacked_max_coeff = []
orig_stacked_min_coeff = []
orig_stacked_mean_coeff = []
gsm_stacked_max_coeff = []
gsm_stacked_min_coeff = []
gsm_stacked_mean_coeff = []
 
for i in range(39):
  mx = np.max(orig_unnormalised_stacked[:,i]) 
  min = np.min(orig_unnormalised_stacked[:,i])
  mn = np.mean(orig_unnormalised_stacked[:,i])
  orig_stacked[:,i] = (orig_unnormalised_stacked[:,i]-mn)/(mx-min)
  #orig_stacked_test[:,i] = (orig_unnormalised_stacked_test[:,i]-mn)/(mx-min)

  orig_stacked_max_coeff.append(mx)
  orig_stacked_min_coeff.append(min)
  orig_stacked_mean_coeff.append(mn)
  
  mx = np.max(gsm_unnormalised_stacked[:,i]) 
  min = np.min(gsm_unnormalised_stacked[:,i])
  mn = np.mean(gsm_unnormalised_stacked[:,i])
  gsm_stacked[:,i] = (gsm_unnormalised_stacked[:,i]-mn)/(mx-min)
  #gsm_stacked_test[:,i] = (gsm_unnormalised_stacked_test[:,i]-mn)/(mx-min)
  
  gsm_stacked_max_coeff.append(mx)
  gsm_stacked_min_coeff.append(min)
  gsm_stacked_mean_coeff.append(mn)  

orig_stacked_max_coeff = np.array(orig_stacked_max_coeff)
orig_stacked_min_coeff = np.array(orig_stacked_min_coeff)
orig_stacked_mean_coeff = np.array(orig_stacked_mean_coeff)

gsm_stacked_max_coeff = np.array(gsm_stacked_max_coeff)
gsm_stacked_min_coeff = np.array(gsm_stacked_min_coeff)
gsm_stacked_mean_coeff = np.array(gsm_stacked_mean_coeff)
err_of_gsm_stacked = gsm_stacked - orig_stacked
#err_of_gsm_stacked_test = sip_stacked_test - orig_stacked_test





**TRAINING LSTM**

In [8]:
min_frames= 145

In [9]:
p=orig_stacked.shape[0]//min_frames
q=orig_stacked.shape[0]%min_frames

normalisaton - x-mean/max-min, 
input = gsm, output = orig

In [10]:
lstm_orig_mfcc = np.reshape(orig_stacked[:-q,:],(p,min_frames,39))
lstm_gsm_mfcc = np.reshape(gsm_stacked[:-q,:],(p,min_frames,39))
#lstm_orig_gsm_err_mfcc = np.reshape(err_of_gsm_stacked[:-q,:],(p,min_frames,39))


In [11]:
#p=orig_stacked_test.shape[0]//min_frames
#q=orig_stacked_test.shape[0]%min_frames

In [12]:
#lstm_orig_mfcc_test = np.reshape(orig_stacked_test[:-q,:],(p,min_frames,39))
#lstm_gsm_mfcc_test = np.reshape(gsm_stacked_test[:-q,:],(p,min_frames,39))
#lstm_orig_gsm_err_mfcc_test = np.reshape(err_of_gsm_stacked_test[:-q,:],(p,min_frames,39))

In [13]:
lstm_gsm_train, lstm_gsm_val, lstm_orig_train, lstm_orig_val = train_test_split(lstm_gsm_mfcc, lstm_orig_mfcc, test_size=0.3)

In [14]:
model3 = tf.keras.Sequential()
model3.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units= min_frames,input_shape = (145, 39),return_sequences=True, activation= 'tanh')))
model3.add(tf.keras.layers.GaussianNoise(0.001))
model3.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units= 145,return_sequences=True, activation= 'tanh'), merge_mode= 'sum'))
model3.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense((39), activation= 'tanh', kernel_regularizer= None)))
model3.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense((39), activation= None)))

In [15]:
los = tf.keras.losses.MeanSquaredError()
initial_learning_rate = 0.01
opt = tf.keras.optimizers.Adam(learning_rate = initial_learning_rate)
model3.compile(loss = los, optimizer= opt, \
              metrics = [tf.keras.metrics.MeanSquaredError(name= 'mse')])
print("Total utterances :", lstm_gsm_train.shape[0])

Total utterances : 9146


In [18]:
num_epoch = 5
inp_batch_size = 100
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=2, min_lr=0.000)
history = model3.fit(lstm_gsm_train,  lstm_orig_train,
                    batch_size= inp_batch_size,
                    epochs= num_epoch,
                    validation_data=(lstm_gsm_val,lstm_orig_val),
                    verbose= 2,
                    callbacks=[reduce_lr]
                   )

Epoch 1/5
92/92 - 6s - loss: 0.0053 - mse: 0.0053 - val_loss: 0.0054 - val_mse: 0.0054 - lr: 1.0000e-05
Epoch 2/5
92/92 - 7s - loss: 0.0053 - mse: 0.0053 - val_loss: 0.0054 - val_mse: 0.0054 - lr: 1.0000e-05
Epoch 3/5
92/92 - 6s - loss: 0.0053 - mse: 0.0053 - val_loss: 0.0054 - val_mse: 0.0054 - lr: 1.0000e-05
Epoch 4/5
92/92 - 6s - loss: 0.0053 - mse: 0.0053 - val_loss: 0.0054 - val_mse: 0.0054 - lr: 1.0000e-06
Epoch 5/5
92/92 - 6s - loss: 0.0053 - mse: 0.0053 - val_loss: 0.0054 - val_mse: 0.0054 - lr: 1.0000e-06


In [22]:
#save_dir = cur_dir+"/sip-gsm-orig-out-performance/"
#model3.save(cur_dir+'/blstm_gsm_orig_model_latest')

In [20]:
#loaded = tf.keras.models.load_model(cur_dir+'/blstm_gsm_orig_model_latest')